In [1]:
import cv2
import mediapipe as mp
import time
import pandas as pd
import tensorflow as tf
import numpy as np
import json

2023-07-19 23:11:58.720323: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 23:11:59.351426: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

pd.set_option('display.max_rows', None)
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()
model = interpreter.get_signature_runner()

selected_colomns = pd.read_json("inference_args.json", orient="index")
selected_colomns = selected_colomns.transpose().values
selected_colomns = np.reshape(selected_colomns, (164,))


# Read Character to Ordinal Encoding Mapping
with open('character_to_prediction_index.json') as json_file:
    CHAR2ORD = json.load(json_file)

# Ordinal to Character Mapping
ORD2CHAR = {j:i for i,j in CHAR2ORD.items()}

# Output Predictions to string
def outputs2phrase(outputs):
    if outputs.ndim == 2:
        outputs = np.argmax(outputs, axis=1)

    return ''.join([ORD2CHAR.get(s, '') for s in outputs])

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
def get_right_hand_landmark_indexes(hand, frame):
    index_list = ['WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP',
                  'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP',
                  'MIDDLE_FINGER_TIP', 'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_DIP', 'RING_FINGER_TIP',
                  'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP']

    right_hand_landmarks = []
    for part in index_list:
        if hand.right_hand_landmarks:
            right_hand_landmarks.append([frame, f"{frame}-right_hand-{index_list.index(part)}", "right_hand", index_list.index(part), hand.right_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].x, hand.right_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].y, hand.right_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].z])

        else:
            right_hand_landmarks.append([frame, f"{frame}-right_hand-{index_list.index(part)}", "right_hand",index_list.index(part),  0, 0, 0])

    return right_hand_landmarks

In [4]:
def get_left_hand_landmark_indexes(hand, frame):
    index_list = ['WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP',
                  'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP',
                  'MIDDLE_FINGER_TIP', 'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_DIP', 'RING_FINGER_TIP',
                  'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP']

    left_hand_landmarks = []
    for part in index_list:
        if hand.left_hand_landmarks:
            left_hand_landmarks.append([frame, f"{frame}-left_hand-{index_list.index(part)}", "left_hand", index_list.index(part), hand.left_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].x, hand.left_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].y, hand.left_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].z])

        else:
            left_hand_landmarks.append([frame, f"{frame}-left_hand-{index_list.index(part)}", "left_hand",index_list.index(part),  0, 0, 0])

    return left_hand_landmarks

In [5]:
def get_post_landmark_indexes(pose, frame):
    index_list = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']


    pose_landmarks = []
    for part in index_list:
        if pose.pose_landmarks:
            pose_landmarks.append([frame, f"{frame}-pose-{index_list.index(part)}", "pose", index_list.index(part), pose.pose_landmarks.landmark[mp_holistic.PoseLandmark(index_list.index(part))].x, pose.pose_landmarks.landmark[mp_holistic.PoseLandmark(index_list.index(part))].y, pose.pose_landmarks.landmark[mp_holistic.PoseLandmark(index_list.index(part))].z])

        else:
            pose_landmarks.append([frame, f"{frame}-pose-{index_list.index(part)}", "pose",index_list.index(part),  0, 0, 0])

    return pose_landmarks

In [6]:
def get_face_landmark_indexes(face, frame):
    face_landmarks = []
    if face.face_landmarks:
        for i in range(len(face.face_landmarks.landmark)):
            face_landmarks.append([frame, f"{frame}-face-{i}", "face", i, face.face_landmarks.landmark[i].x,
                                   face.face_landmarks.landmark[i].y, face.face_landmarks.landmark[i].z])
    else:
        for i in range(0, 468):
            face_landmarks.append([frame, f"{frame}-face-{i}", "face", i, 0, 0, 0])
    return face_landmarks

In [7]:
def Mediapipe_holistic():
    count = 0
    data = []
    cap = cv2.VideoCapture("WAIT.mp4")
    #cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        while cap.isOpened():

            success, image = cap.read()

            start = time.time()

            if image is not None:
                img_h, img_w, img_c = image.shape

            # Convert the BGR image to RGB.
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
                image.flags.writeable = False

            # Process the image and find hands
                results = hands.process(image)

                image.flags.writeable = True

            # Draw the hand annotations on the image.
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


                mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

                end = time.time()
                totalTime = end - start

                fps = 1 / totalTime
                count = count + 1

                data.append(get_right_hand_landmark_indexes(results, count))
                data.append(get_left_hand_landmark_indexes(results,count))
                data.append(get_post_landmark_indexes(results, count))
                data.append(get_face_landmark_indexes(results, count))
                cv2.putText(image, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2)

                #cv2.imshow('MediaPipe Hands', image)

                #if cv2.waitKey(5) & 0xFF == 27:
                  #break
            else:
                cap.release()

            #for live video
            # cv2.putText(image, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2)
            #
            # cv2.imshow('MediaPipe Hands', image)
            #
            # if cv2.waitKey(5) & 0xFF == 27:
            #   break
    return data

In [8]:
data = Mediapipe_holistic()
flat_list = [item for sublist in data for item in sublist]
df = pd.DataFrame(flat_list, columns=['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z'])

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [9]:
df.head()

,frame,row_id,type,landmark_index,x,y,z
0,1,1-right_hand-0,right_hand,0,0.0,0.0,0.0
1,1,1-right_hand-1,right_hand,1,0.0,0.0,0.0
2,1,1-right_hand-2,right_hand,2,0.0,0.0,0.0
3,1,1-right_hand-3,right_hand,3,0.0,0.0,0.0
4,1,1-right_hand-4,right_hand,4,0.0,0.0,0.0


In [10]:
newdf = pd.read_parquet("one_video.parquet")

In [11]:
newdf.columns.tolist()
with open('newdf_columns.txt', 'w') as file:
    file.write('\n'.join(newdf.columns.tolist()))

In [12]:
#newdf.head()

In [13]:
# Reshape the DataFrame using pivot_table
df = pd.DataFrame({'landmark_index': pd.to_numeric(df['landmark_index'], errors='coerce'),
                   'frame': pd.to_numeric(df['frame'], errors='coerce'),
                   'type': df['type'].astype(str),
                   'x': df['x'],
                   'y': df['y'],
                   'z': df['z']})

In [14]:
df.columns

Index(['landmark_index', 'frame', 'type', 'x', 'y', 'z'], dtype='object')

In [15]:
df_pivot = df.pivot_table(index='frame', columns=['type', 'landmark_index'])

In [16]:
df_pivot

x                                                     
type                face                                                     
landmark_index        0         1         2         3         4         5    
frame                                                                        
1               0.539060  0.541514  0.539884  0.537000  0.541945  0.541833  \
2               0.539790  0.542298  0.540561  0.537582  0.542723  0.542572   
3               0.539625  0.541992  0.540317  0.537236  0.542394  0.542212   
4               0.539331  0.541843  0.540116  0.537124  0.542256  0.542101   
5               0.539128  0.541710  0.539934  0.536963  0.542130  0.541964   
6               0.538897  0.541431  0.539663  0.536635  0.541836  0.541654   
7               0.538514  0.541069  0.539323  0.536370  0.541480  0.541325   
8               0.538284  0.540875  0.539094  0.536150  0.541291  0.541125   
9               0.538213  0.540914  0.539048  0.536184  0.541347  0.541167   
10              0.538083  0.540733  0.538885  0.536011  0.541163  0.540985   
11              0.538358  0.540845  0.539039  0.536071  0.541247  0.541032   
12              0.538592  0.540965  0.539211  0.536091  0.541327  0.541069   
13              0.538970  0.541376  0.539583  0.536441  0.541738  0.541461   
14              0.539218  0.541658  0.539820  0.536711  0.542029  0.541746   
15              0.539219  0.541675  0.539826  0.536731  0.542052  0.541771   
16              0.538909  0.541583  0.539639  0.536770  0.542010  0.541789   
17              0.538549  0.541268  0.539343  0.536589  0.541718  0.541554   
18              0.537977  0.540822  0.538831  0.536110  0.541273  0.541091   
19              0.537476  0.540247  0.538296  0.535531  0.540692  0.540511   
20              0.537007  0.539744  0.537801  0.535006  0.540184  0.539995   
21              0.536130  0.538612  0.536853  0.533989  0.539039  0.538903   
22              0.535823  0.538449  0.536600  0.533721  0.538868  0.538694   
23              0.535459  0.537983  0.536186  0.533267  0.538393  0.538224   
24              0.535197  0.537701  0.535944  0.533207  0.538150  0.538043   
25              0.534918  0.537234  0.535587  0.532772  0.537662  0.537561   
26              0.533901  0.536131  0.534618  0.531876  0.536564  0.536542   
27              0.533133  0.535322  0.533850  0.531071  0.535757  0.535765   
28              0.532559  0.534586  0.533206  0.530433  0.535013  0.535045   
29              0.531721  0.533811  0.532467  0.529668  0.534236  0.534303   
30              0.531558  0.533703  0.532301  0.529570  0.534142  0.534200   
31              0.531342  0.533433  0.532083  0.529337  0.533866  0.533936   
32              0.531017  0.533110  0.531789  0.529190  0.533573  0.533708   
33              0.531031  0.533025  0.531774  0.529093  0.533467  0.533596   
34              0.531510  0.533301  0.532083  0.529308  0.533708  0.533790   
35              0.531557  0.533254  0.532083  0.529269  0.533647  0.533719   
36              0.531661  0.533350  0.532176  0.529336  0.533755  0.533825   
37              0.531374  0.533097  0.531945  0.529189  0.533523  0.533639   
38              0.531458  0.533170  0.532000  0.529255  0.533601  0.533710   
39              0.531324  0.533031  0.531855  0.529078  0.533451  0.533537   
40              0.530998  0.532678  0.531559  0.528764  0.533091  0.533198   
41              0.530905  0.532565  0.531459  0.528669  0.532978  0.533085   
42              0.530754  0.532473  0.531328  0.528487  0.532874  0.532955   
43              0.530244  0.531911  0.530835  0.527972  0.532308  0.532418   
44              0.530001  0.531635  0.530577  0.527630  0.532012  0.532102   
45              0.529408  0.531097  0.529998  0.526926  0.531451  0.531498   
46              0.528431  0.529998  0.529009  0.525965  0.530364  0.530468   
47              0.527584  0.529138  0.528178  0.525097  0.529495  0.529618   
48              0.527008  0.528489  0.527544 

In [17]:
['{}_{}_{}'.format(pos, type_val, point) for pos, type_val, point in df_pivot.columns]

['x_face_0',
 'x_face_1',
 'x_face_2',
 'x_face_3',
 'x_face_4',
 'x_face_5',
 'x_face_6',
 'x_face_7',
 'x_face_8',
 'x_face_9',
 'x_face_10',
 'x_face_11',
 'x_face_12',
 'x_face_13',
 'x_face_14',
 'x_face_15',
 'x_face_16',
 'x_face_17',
 'x_face_18',
 'x_face_19',
 'x_face_20',
 'x_face_21',
 'x_face_22',
 'x_face_23',
 'x_face_24',
 'x_face_25',
 'x_face_26',
 'x_face_27',
 'x_face_28',
 'x_face_29',
 'x_face_30',
 'x_face_31',
 'x_face_32',
 'x_face_33',
 'x_face_34',
 'x_face_35',
 'x_face_36',
 'x_face_37',
 'x_face_38',
 'x_face_39',
 'x_face_40',
 'x_face_41',
 'x_face_42',
 'x_face_43',
 'x_face_44',
 'x_face_45',
 'x_face_46',
 'x_face_47',
 'x_face_48',
 'x_face_49',
 'x_face_50',
 'x_face_51',
 'x_face_52',
 'x_face_53',
 'x_face_54',
 'x_face_55',
 'x_face_56',
 'x_face_57',
 'x_face_58',
 'x_face_59',
 'x_face_60',
 'x_face_61',
 'x_face_62',
 'x_face_63',
 'x_face_64',
 'x_face_65',
 'x_face_66',
 'x_face_67',
 'x_face_68',
 'x_face_69',
 'x_face_70',
 'x_face_71',
 '

In [18]:

# Flatten the MultiIndex column names
#df_pivot.columns = ['{}_{}_{}'.format(pos, type_val, point) for pos, type_val, point in df_pivot.columns]

In [19]:
df_pivot.columns

MultiIndex([('x',       'face',  0),
            ('x',       'face',  1),
            ('x',       'face',  2),
            ('x',       'face',  3),
            ('x',       'face',  4),
            ('x',       'face',  5),
            ('x',       'face',  6),
            ('x',       'face',  7),
            ('x',       'face',  8),
            ('x',       'face',  9),
            ...
            ('z', 'right_hand', 11),
            ('z', 'right_hand', 12),
            ('z', 'right_hand', 13),
            ('z', 'right_hand', 14),
            ('z', 'right_hand', 15),
            ('z', 'right_hand', 16),
            ('z', 'right_hand', 17),
            ('z', 'right_hand', 18),
            ('z', 'right_hand', 19),
            ('z', 'right_hand', 20)],
           names=[None, 'type', 'landmark_index'], length=1629)

In [20]:
# Create the final DataFrame
final_df = pd.DataFrame(df_pivot.to_records(), columns=['{}_{}_{}'.format(pos, type_val, point) for pos, type_val, point in df_pivot.columns])

In [21]:
final_df

,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,x_face_9,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
common_columns = final_df.columns.intersection(newdf.columns)


In [23]:
final_df

,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,x_face_9,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
columns_to_remove = list(set(final_df.columns) - set(newdf.columns))
with open('removed_columns.txt', 'w') as file:
    file.write('\n'.join(columns_to_remove))

In [25]:
file = open("removed_columns.txt")

In [26]:
with open('removed_columns.txt', 'r') as file:
    removed_columns = [column.strip() for column in file.readlines()]

# Print the removed columns
len(removed_columns)

1465

In [33]:
removed_columns2 = []

In [34]:
[removed_columns2 for i in removed_columns if i == 'frame']

[]

In [27]:
final_df.drop(columns=removed_columns, inplace=True)

In [28]:
final_df

,x_face_0,x_face_13,x_face_14,x_face_17,x_face_37,x_face_39,x_face_40,x_face_61,x_face_78,x_face_80,...,y_right_hand_11,y_right_hand_12,y_right_hand_13,y_right_hand_14,y_right_hand_15,y_right_hand_16,y_right_hand_17,y_right_hand_18,y_right_hand_19,y_right_hand_20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#cast to float32
final_df = final_df.astype('float32')

In [30]:
demo_output = model(inputs=final_df.values)['outputs']
print(f'phrase decoded: {outputs2phrase(demo_output)}')

phrase decoded: 2113 colon
